# Surrounding Venues for Each Arcade

__Skip to the Good Stuff__

[Importing Libraries](#Importing-Libraries)

[Loading Arcade DataFrame](#Loading-Arcade-DataFrame)

[Connecting to FourSquare](#Connecting-to-FourSquare)

[Making Surrounding Venues DataFrame](#Making-Surrounding-Venues-DataFrame)

[Dropping the Nulls](#Dropping-the-Nulls)

[Sorting by Arcade and Distance](#Sorting-by-Arcade-and-Distance)

[Ensuring Unique Venues](#Ensuring-Unique-Venues)

[Pulling the Ratings](#Pulling-the-Ratings)

### Importing Libraries

In [26]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation

!pip install geopy
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 

! pip install folium==0.5.0
import folium # plotting library

import json

pd.set_option('display.max_rows', None) #To view the full dataframe

print('Folium installed')
print('Libraries imported.')

Folium installed
Libraries imported.


### Loading Arcade DataFrame

In [2]:
dallas_arcades = pd.read_csv('Arcade_final_clean.csv')
dallas_arcades = dallas_arcades.drop('Unnamed: 0', axis = 1)
print(len(dallas_arcades))
dallas_arcades

8


,Hood,Venue_ID,Venue_Name,Venue_Distance,Venue_Address,Venue_Latitude,Venue_Longitude,Category_ID,Category_Name,Venue_Rating
0,Swiss Avenue,4ab03c84f964a520996620e3,Arcade Bar,515,1825 Abrams Rd,32.812223,-96.753279,4bf58dd8d48988d116941735,Bar,Not Rated
1,Grapevine,4c538a9530f92d7fad2af0b8,Northern Lights Arcade at Great Wolf Lodge,905,100 Great Wolf Dr,32.932259,-97.039587,4bf58dd8d48988d1e1931735,Arcade,Not Rated
2,Bluffview,4e18b36c183880768f442784,The Arcade,1687,No Address,32.970596,-96.925092,4bf58dd8d48988d1e1931735,Arcade,Not Rated
3,Rockwall,4f32bb1b19836c91c7f4450e,Spin and Win Arcade,840,10219 Plano Rd,32.889265,-96.703578,4d4b7105d754a06376d81259,Nightlife Spot,Not Rated
4,Coppell,4f551efde4b0336536ffd4a6,Noah's Arcade,1520,No Address,32.964215,-97.027868,4bf58dd8d48988d1e1931735,Arcade,Not Rated
5,Arts District,50d39fe2e4b06054da0fc08b,BBs Arcade,905,412 N Tyler St,32.748692,-96.837542,4bf58dd8d48988d1e1931735,Arcade,Not Rated
6,Plano,56631de8498e2afbdc225b72,Free Play Arcade,1305,1730 E Belt Line Rd,32.943863,-96.693342,4bf58dd8d48988d1e1931735,Arcade,Not Rated
7,Sachse,574931cd498ec191ec0e8f52,Gonzo's Rolling Arcade,1665,No Address,32.961290,-96.594286,4bf58dd8d48988d1e1931735,Arcade,Not Rated


### Connecting to FourSquare

In [3]:
CLIENT_ID = '4TJ1SLTDO1KADL553GXCDXOJXZONZYTBIRQMVBQ32JYAU3EA' # your Foursquare ID
CLIENT_SECRET = 'TC1N45PW5LSGVLIZ02DDPA2JOMWB2I3BA33OPX55VW2LBKBF' # your Foursquare Secret
ACCESS_TOKEN = 'NS3WWURVAWWAGWSTUXQDQCS0150IPNQIYNHVE5JM1OSX2CAV' # your FourSquare Access Token

In [4]:
VERSION = '20180604'
LIMIT = 20

### Making Surrounding Venues DataFrame

In [5]:
#assign variables for the search
search_query = ''
radius = 50
print(search_query + ' .... OK!')

 .... OK!


In [6]:
surrounding_venues = pd.DataFrame(columns = ['Arcade','Venue_ID','Venue_Name','Venue_Distance','Venue_Address','Venue_Latitude','Venue_Longitude','Category_ID','Category_Name'])
surrounding_venues

,Arcade,Venue_ID,Venue_Name,Venue_Distance,Venue_Address,Venue_Latitude,Venue_Longitude,Category_ID,Category_Name


In [7]:
for index, arcade in enumerate(dallas_arcades['Venue_ID']):
    try:
        arcade_name = dallas_arcades['Venue_Name'][index]
        latitude = dallas_arcades['Venue_Latitude'][index]
        longitude = dallas_arcades['Venue_Longitude'][index]
        url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&oauth_token={}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude,ACCESS_TOKEN, VERSION, search_query, radius, LIMIT)
        results = requests.get(url).json()
    except:
        continue
    try:
        res = results['response']['venues']
    except:
        print('No response')
    for dictionary in results['response']['venues']:
        try:
            ident= dictionary['id']
        except:
            ident= 'No ID'
        try:
            name = dictionary['name']
        except:
            name = 'No Name'
        try:
            ven_add = dictionary['location']['address']
        except:
            ven_add = 'No Address'
        try:
            ven_lat = dictionary['location']['lat']
        except:
            ven_lat = 'No Latitude'
        try:
            ven_lng = dictionary['location']['lng']
        except:
            ven_lng = 'No Longitude'
        try:
            ven_dist = dictionary['location']['distance']
        except:
            ven_dist = 'No Distance'
        try:
            for cat in dictionary['categories']:
                try:
                    cat_id= cat['id']
                except:
                    cat_id= 'No ID'
                try:
                    cat_name = cat['name']
                except:
                    cat_name = 'No Name'
        except:
            continue
        try:
            surrounding_venues = surrounding_venues.append({'Venue_Distance':ven_dist,'Arcade':arcade_name, 'Venue_ID':ident, 'Venue_Name':name,'Venue_Address':ven_add,'Venue_Latitude':ven_lat,'Venue_Longitude':ven_lng,'Category_ID':cat_id,'Category_Name':cat_name}, ignore_index=True)
        except:
            surrounding_venues = surrounding_venues.append({'Arcade': 'Null'}, ignore_index=True)

In [8]:
print(len(surrounding_venues))
surrounding_venues.head()

160


,Arcade,Venue_ID,Venue_Name,Venue_Distance,Venue_Address,Venue_Latitude,Venue_Longitude,Category_ID,Category_Name
0,Arcade Bar,40e0b100f964a520ea071fe3,Balcony Club,10,1825 Abrams Rd Ste B,32.812265,-96.753374,4bf58dd8d48988d1e7931735,Jazz Club
1,Arcade Bar,4ab03c84f964a520996620e3,Arcade Bar,0,1825 Abrams Rd,32.812223,-96.753279,4bf58dd8d48988d116941735,Bar
2,Arcade Bar,4fbbb1ace4b0c28e72dad0e7,Mi Cocina,23,6332 La Vista Dr,32.812305,-96.753508,4bf58dd8d48988d1c1941735,Mexican Restaurant
3,Arcade Bar,4234d400f964a52012201fe3,Lakewood Theater,6,1825 Abrams Rd,32.812165,-96.753258,4bf58dd8d48988d1f2931735,Performing Arts Venue
4,Arcade Bar,5d6c6950716af5000829b0dc,Bowkski’s,46,1825 Abrams Rd,32.811833,-96.753104,4bf58dd8d48988d1e4931735,Bowling Alley


In [9]:
surrounding_venues.to_csv('surrounding_raw.csv')

### Dropping the Nulls

In [10]:
surrounding_venues = pd.read_csv('surrounding_raw.csv')
surrounding_venues = surrounding_venues.drop('Unnamed: 0', axis= 1)
print(len(surrounding_venues))
surrounding_venues.head()

160


,Arcade,Venue_ID,Venue_Name,Venue_Distance,Venue_Address,Venue_Latitude,Venue_Longitude,Category_ID,Category_Name
0,Arcade Bar,40e0b100f964a520ea071fe3,Balcony Club,10,1825 Abrams Rd Ste B,32.812265,-96.753374,4bf58dd8d48988d1e7931735,Jazz Club
1,Arcade Bar,4ab03c84f964a520996620e3,Arcade Bar,0,1825 Abrams Rd,32.812223,-96.753279,4bf58dd8d48988d116941735,Bar
2,Arcade Bar,4fbbb1ace4b0c28e72dad0e7,Mi Cocina,23,6332 La Vista Dr,32.812305,-96.753508,4bf58dd8d48988d1c1941735,Mexican Restaurant
3,Arcade Bar,4234d400f964a52012201fe3,Lakewood Theater,6,1825 Abrams Rd,32.812165,-96.753258,4bf58dd8d48988d1f2931735,Performing Arts Venue
4,Arcade Bar,5d6c6950716af5000829b0dc,Bowkski’s,46,1825 Abrams Rd,32.811833,-96.753104,4bf58dd8d48988d1e4931735,Bowling Alley


In [11]:
surrounding_venues = surrounding_venues[surrounding_venues != 'Null']
surrounding_venues = surrounding_venues[surrounding_venues != '[]']
surrounding_venues = surrounding_venues.dropna()
print(len(surrounding_venues))
surrounding_venues = surrounding_venues.reset_index()
surrounding_venues = surrounding_venues.drop(['index'], axis = 1)
surrounding_venues.head(20)

160


,Arcade,Venue_ID,Venue_Name,Venue_Distance,Venue_Address,Venue_Latitude,Venue_Longitude,Category_ID,Category_Name
0,Arcade Bar,40e0b100f964a520ea071fe3,Balcony Club,10,1825 Abrams Rd Ste B,32.812265,-96.753374,4bf58dd8d48988d1e7931735,Jazz Club
1,Arcade Bar,4ab03c84f964a520996620e3,Arcade Bar,0,1825 Abrams Rd,32.812223,-96.753279,4bf58dd8d48988d116941735,Bar
2,Arcade Bar,4fbbb1ace4b0c28e72dad0e7,Mi Cocina,23,6332 La Vista Dr,32.812305,-96.753508,4bf58dd8d48988d1c1941735,Mexican Restaurant
3,Arcade Bar,4234d400f964a52012201fe3,Lakewood Theater,6,1825 Abrams Rd,32.812165,-96.753258,4bf58dd8d48988d1f2931735,Performing Arts Venue
4,Arcade Bar,5d6c6950716af5000829b0dc,Bowkski’s,46,1825 Abrams Rd,32.811833,-96.753104,4bf58dd8d48988d1e4931735,Bowling Alley
5,Arcade Bar,5d6efe9be1edd6000826d267,Bowlski’s,78,No Address,32.811523,-96.753198,4bf58dd8d48988d1e4931735,Bowling Alley
6,Arcade Bar,4dd6a1d3b0fb8af38071e075,Campagna & Assoc Realtors,60,726 Paulus Ave,32.811976,-96.753856,4bf58dd8d48988d1e4931735,Bowling Alley
7,Arcade Bar,4f92e9c9e4b00fe65eb772de,Harrell Park,132,No Address,32.812688,-96.751974,4bf58dd8d48988d163941735,Park
8,Arcade Bar,4abfcfedf964a520649220e3,Starbucks,78,6312 La Vista,32.812518,-96.754035,4bf58dd8d48988d1e0931735,Coffee Shop
9,Arcade Bar,4bccc6e0511f9521d49eb3c7,The UPS Store,2073,"7324 Gaston Ave, Ste 124",32.811581,-96.731124,52f2ab2ebcbc57f1066b8b1f,Shipping Store


In [12]:
count = -1
distance_drop_list = list()
for dist in surrounding_venues['Venue_Distance']:
    count = count + 1
    if dist == 0:
        distance_drop_list.append(count)
print(len(distance_drop_list))

6


In [13]:
surrounding_venues = surrounding_venues.drop(index = distance_drop_list)
print(len(surrounding_venues))
surrounding_venues.head()

154


,Arcade,Venue_ID,Venue_Name,Venue_Distance,Venue_Address,Venue_Latitude,Venue_Longitude,Category_ID,Category_Name
0,Arcade Bar,40e0b100f964a520ea071fe3,Balcony Club,10,1825 Abrams Rd Ste B,32.812265,-96.753374,4bf58dd8d48988d1e7931735,Jazz Club
2,Arcade Bar,4fbbb1ace4b0c28e72dad0e7,Mi Cocina,23,6332 La Vista Dr,32.812305,-96.753508,4bf58dd8d48988d1c1941735,Mexican Restaurant
3,Arcade Bar,4234d400f964a52012201fe3,Lakewood Theater,6,1825 Abrams Rd,32.812165,-96.753258,4bf58dd8d48988d1f2931735,Performing Arts Venue
4,Arcade Bar,5d6c6950716af5000829b0dc,Bowkski’s,46,1825 Abrams Rd,32.811833,-96.753104,4bf58dd8d48988d1e4931735,Bowling Alley
5,Arcade Bar,5d6efe9be1edd6000826d267,Bowlski’s,78,No Address,32.811523,-96.753198,4bf58dd8d48988d1e4931735,Bowling Alley


### Sorting by Arcade and Venue Distance

In [14]:
surrounding_venues = surrounding_venues.sort_values(['Arcade', 'Venue_Distance'],ascending = True)
surrounding_venues.head(10)

,Arcade,Venue_ID,Venue_Name,Venue_Distance,Venue_Address,Venue_Latitude,Venue_Longitude,Category_ID,Category_Name
3,Arcade Bar,4234d400f964a52012201fe3,Lakewood Theater,6,1825 Abrams Rd,32.812165,-96.753258,4bf58dd8d48988d1f2931735,Performing Arts Venue
0,Arcade Bar,40e0b100f964a520ea071fe3,Balcony Club,10,1825 Abrams Rd Ste B,32.812265,-96.753374,4bf58dd8d48988d1e7931735,Jazz Club
2,Arcade Bar,4fbbb1ace4b0c28e72dad0e7,Mi Cocina,23,6332 La Vista Dr,32.812305,-96.753508,4bf58dd8d48988d1c1941735,Mexican Restaurant
4,Arcade Bar,5d6c6950716af5000829b0dc,Bowkski’s,46,1825 Abrams Rd,32.811833,-96.753104,4bf58dd8d48988d1e4931735,Bowling Alley
6,Arcade Bar,4dd6a1d3b0fb8af38071e075,Campagna & Assoc Realtors,60,726 Paulus Ave,32.811976,-96.753856,4bf58dd8d48988d1e4931735,Bowling Alley
5,Arcade Bar,5d6efe9be1edd6000826d267,Bowlski’s,78,No Address,32.811523,-96.753198,4bf58dd8d48988d1e4931735,Bowling Alley
8,Arcade Bar,4abfcfedf964a520649220e3,Starbucks,78,6312 La Vista,32.812518,-96.754035,4bf58dd8d48988d1e0931735,Coffee Shop
17,Arcade Bar,4b987e24f964a520854435e3,Jack in the Box,92,6308 Gaston Ave,32.812340,-96.754260,4bf58dd8d48988d16e941735,Fast Food Restaurant
10,Arcade Bar,5dd892fc1588c30008720e28,Waya Japanese Izakaya,100,6334 Gaston Ave,32.813103,-96.753494,4bf58dd8d48988d111941735,Japanese Restaurant
11,Arcade Bar,4bd623705631c9b6266ea530,City Nails,114,6406 Gaston Ave,32.813255,-96.753270,4bf58dd8d48988d10c951735,Cosmetics Shop


In [15]:
surrounding_venues.to_csv('surrounding_venues_sorted.csv')

### Ensuring Unique Venues

In [16]:
surrounding_sorted = pd.read_csv('surrounding_venues_sorted.csv')
surrounding_sorted = surrounding_sorted.drop('Unnamed: 0', axis= 1)
print(len(surrounding_sorted))
surrounding_sorted.head(20)

154


,Arcade,Venue_ID,Venue_Name,Venue_Distance,Venue_Address,Venue_Latitude,Venue_Longitude,Category_ID,Category_Name
0,Arcade Bar,4234d400f964a52012201fe3,Lakewood Theater,6,1825 Abrams Rd,32.812165,-96.753258,4bf58dd8d48988d1f2931735,Performing Arts Venue
1,Arcade Bar,40e0b100f964a520ea071fe3,Balcony Club,10,1825 Abrams Rd Ste B,32.812265,-96.753374,4bf58dd8d48988d1e7931735,Jazz Club
2,Arcade Bar,4fbbb1ace4b0c28e72dad0e7,Mi Cocina,23,6332 La Vista Dr,32.812305,-96.753508,4bf58dd8d48988d1c1941735,Mexican Restaurant
3,Arcade Bar,5d6c6950716af5000829b0dc,Bowkski’s,46,1825 Abrams Rd,32.811833,-96.753104,4bf58dd8d48988d1e4931735,Bowling Alley
4,Arcade Bar,4dd6a1d3b0fb8af38071e075,Campagna & Assoc Realtors,60,726 Paulus Ave,32.811976,-96.753856,4bf58dd8d48988d1e4931735,Bowling Alley
5,Arcade Bar,5d6efe9be1edd6000826d267,Bowlski’s,78,No Address,32.811523,-96.753198,4bf58dd8d48988d1e4931735,Bowling Alley
6,Arcade Bar,4abfcfedf964a520649220e3,Starbucks,78,6312 La Vista,32.812518,-96.754035,4bf58dd8d48988d1e0931735,Coffee Shop
7,Arcade Bar,4b987e24f964a520854435e3,Jack in the Box,92,6308 Gaston Ave,32.812340,-96.754260,4bf58dd8d48988d16e941735,Fast Food Restaurant
8,Arcade Bar,5dd892fc1588c30008720e28,Waya Japanese Izakaya,100,6334 Gaston Ave,32.813103,-96.753494,4bf58dd8d48988d111941735,Japanese Restaurant
9,Arcade Bar,4bd623705631c9b6266ea530,City Nails,114,6406 Gaston Ave,32.813255,-96.753270,4bf58dd8d48988d10c951735,Cosmetics Shop


### Pulling the Ratings

In [17]:
count = -1
rating_list = list()
for ident in surrounding_sorted['Venue_ID']:
    count = count +1
    venue_id = surrounding_sorted['Venue_ID'][count]
    url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&oauth_token={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET,ACCESS_TOKEN, VERSION)
    result = requests.get(url).json()
    try:
        rating = result['response']['venue']['rating']
    except:
        rating = 'Not Rated'
    rating_list.append(rating)
rating_list

[8.2,
 8.2,
 8.0,
 'Not Rated',
 'Not Rated',
 'Not Rated',
 7.9,
 5.8,
 'Not Rated',
 8.1,
 8.2,
 'Not Rated',
 'Not Rated',
 'Not Rated',
 'Not Rated',
 7.4,
 6.4,
 8.2,
 6.7,
 'Not Rated',
 'Not Rated',
 'Not Rated',
 'Not Rated',
 'Not Rated',
 'Not Rated',
 'Not Rated',
 'Not Rated',
 'Not Rated',
 'Not Rated',
 'Not Rated',
 'Not Rated',
 'Not Rated',
 'Not Rated',
 'Not Rated',
 'Not Rated',
 'Not Rated',
 5.3,
 'Not Rated',
 'Not Rated',
 'Not Rated',
 6.2,
 6.6,
 'Not Rated',
 7.2,
 6.4,
 7.2,
 'Not Rated',
 7.1,
 7.2,
 5.2,
 5.8,
 'Not Rated',
 5.7,
 7.2,
 'Not Rated',
 7.1,
 5.4,
 'Not Rated',
 'Not Rated',
 'Not Rated',
 'Not Rated',
 'Not Rated',
 'Not Rated',
 'Not Rated',
 'Not Rated',
 'Not Rated',
 'Not Rated',
 'Not Rated',
 7.1,
 'Not Rated',
 'Not Rated',
 'Not Rated',
 6.5,
 'Not Rated',
 'Not Rated',
 'Not Rated',
 6.1,
 'Not Rated',
 'Not Rated',
 'Not Rated',
 'Not Rated',
 'Not Rated',
 'Not Rated',
 'Not Rated',
 'Not Rated',
 'Not Rated',
 'Not Rated',
 'Not 

In [18]:
surrounding_final = surrounding_sorted.assign(Venue_Rating= rating_list)
surrounding_final.head()

,Arcade,Venue_ID,Venue_Name,Venue_Distance,Venue_Address,Venue_Latitude,Venue_Longitude,Category_ID,Category_Name,Venue_Rating
0,Arcade Bar,4234d400f964a52012201fe3,Lakewood Theater,6,1825 Abrams Rd,32.812165,-96.753258,4bf58dd8d48988d1f2931735,Performing Arts Venue,8.2
1,Arcade Bar,40e0b100f964a520ea071fe3,Balcony Club,10,1825 Abrams Rd Ste B,32.812265,-96.753374,4bf58dd8d48988d1e7931735,Jazz Club,8.2
2,Arcade Bar,4fbbb1ace4b0c28e72dad0e7,Mi Cocina,23,6332 La Vista Dr,32.812305,-96.753508,4bf58dd8d48988d1c1941735,Mexican Restaurant,8
3,Arcade Bar,5d6c6950716af5000829b0dc,Bowkski’s,46,1825 Abrams Rd,32.811833,-96.753104,4bf58dd8d48988d1e4931735,Bowling Alley,Not Rated
4,Arcade Bar,4dd6a1d3b0fb8af38071e075,Campagna & Assoc Realtors,60,726 Paulus Ave,32.811976,-96.753856,4bf58dd8d48988d1e4931735,Bowling Alley,Not Rated


In [19]:
counts = dict()
for category in surrounding_final['Category_Name']:
    counts[category]= counts.get(category, 0) + 1

In [20]:
counts

{'Performing Arts Venue': 1,
 'Jazz Club': 1,
 'Mexican Restaurant': 2,
 'Bowling Alley': 3,
 'Coffee Shop': 4,
 'Fast Food Restaurant': 3,
 'Japanese Restaurant': 1,
 'Cosmetics Shop': 1,
 'Restaurant': 1,
 'Juice Bar': 1,
 'Gift Shop': 3,
 'Park': 2,
 'Office': 8,
 'Gym / Fitness Center': 1,
 'Bank': 1,
 'Beer Store': 1,
 'Shipping Store': 1,
 'Art Gallery': 6,
 'Flower Shop': 2,
 'Arts & Entertainment': 1,
 'Salon / Barbershop': 3,
 'Bike Shop': 1,
 'Furniture / Home Store': 3,
 'Boutique': 3,
 'Vegetarian / Vegan Restaurant': 1,
 'Design Studio': 1,
 "Doctor's Office": 2,
 'Southern / Soul Food Restaurant': 1,
 'Martial Arts School': 1,
 'Bar': 2,
 'Discount Store': 3,
 'Ice Cream Shop': 1,
 'Hardware Store': 2,
 'Sports Bar': 2,
 'Pub': 1,
 'Thrift / Vintage Store': 1,
 'Fried Chicken Joint': 1,
 'Pizza Place': 1,
 'Vietnamese Restaurant': 1,
 'Big Box Store': 1,
 'American Restaurant': 2,
 'Elementary School': 2,
 'Bus Line': 1,
 'Assisted Living': 1,
 'Korean Restaurant': 1,
 'L

In [21]:
surrounding_final.to_csv('surrounding_final.csv')

### Prepping for Clustering

In [27]:
surrounding_split = pd.read_csv('surrounding_final.csv')
surrounding_split = surrounding_split.drop('Unnamed: 0', axis = 1)
print(len(surrounding_split))
surrounding_split.head()

154


,Arcade,Venue_ID,Venue_Name,Venue_Distance,Venue_Address,Venue_Latitude,Venue_Longitude,Category_ID,Category_Name,Venue_Rating
0,Arcade Bar,4234d400f964a52012201fe3,Lakewood Theater,6,1825 Abrams Rd,32.812165,-96.753258,4bf58dd8d48988d1f2931735,Performing Arts Venue,8.2
1,Arcade Bar,40e0b100f964a520ea071fe3,Balcony Club,10,1825 Abrams Rd Ste B,32.812265,-96.753374,4bf58dd8d48988d1e7931735,Jazz Club,8.2
2,Arcade Bar,4fbbb1ace4b0c28e72dad0e7,Mi Cocina,23,6332 La Vista Dr,32.812305,-96.753508,4bf58dd8d48988d1c1941735,Mexican Restaurant,8.0
3,Arcade Bar,5d6c6950716af5000829b0dc,Bowkski’s,46,1825 Abrams Rd,32.811833,-96.753104,4bf58dd8d48988d1e4931735,Bowling Alley,Not Rated
4,Arcade Bar,4dd6a1d3b0fb8af38071e075,Campagna & Assoc Realtors,60,726 Paulus Ave,32.811976,-96.753856,4bf58dd8d48988d1e4931735,Bowling Alley,Not Rated


In [28]:
venue_clustering_df = pd.DataFrame(surrounding_split[['Venue_Distance', 'Category_ID', 'Category_Name', 'Venue_Rating']])
venue_clustering_df.head()

,Venue_Distance,Category_ID,Category_Name,Venue_Rating
0,6,4bf58dd8d48988d1f2931735,Performing Arts Venue,8.2
1,10,4bf58dd8d48988d1e7931735,Jazz Club,8.2
2,23,4bf58dd8d48988d1c1941735,Mexican Restaurant,8.0
3,46,4bf58dd8d48988d1e4931735,Bowling Alley,Not Rated
4,60,4bf58dd8d48988d1e4931735,Bowling Alley,Not Rated


In [29]:
venue_clustering_df.to_csv('dallas_clustering_category.csv')

In [30]:
arcade_venue_df = pd.DataFrame(surrounding_split[['Arcade', 'Venue_ID', 'Venue_Name', 'Venue_Address']])
arcade_venue_df.insert(0, 'City', 'Null')
arcade_venue_df = arcade_venue_df.assign(City = 'Dallas, TX')
arcade_venue_df.head()

,City,Arcade,Venue_ID,Venue_Name,Venue_Address
0,"Dallas, TX",Arcade Bar,4234d400f964a52012201fe3,Lakewood Theater,1825 Abrams Rd
1,"Dallas, TX",Arcade Bar,40e0b100f964a520ea071fe3,Balcony Club,1825 Abrams Rd Ste B
2,"Dallas, TX",Arcade Bar,4fbbb1ace4b0c28e72dad0e7,Mi Cocina,6332 La Vista Dr
3,"Dallas, TX",Arcade Bar,5d6c6950716af5000829b0dc,Bowkski’s,1825 Abrams Rd
4,"Dallas, TX",Arcade Bar,4dd6a1d3b0fb8af38071e075,Campagna & Assoc Realtors,726 Paulus Ave


In [32]:
arcade_venue_df.to_csv('dallas_no_cluster_arcade_data.csv')